In [1]:
import boto3

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()



In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/superradiance'.format(account, region)



In [10]:
superradiance = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://sagemaker-kessle31/superradiance/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

superradiance.fit("s3://sagemaker-kessle31/superradiance/initial_state")

INFO:sagemaker:Creating training-job with name: superradiance-2018-01-16-21-01-37-517


......................................................
Starting the training.
MS1
MS2
MS3
MS4
2018-01-16 21:05:59.457534: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
step
step
step
step
step
step
step
step
step
step
Training complete.
===== Job Complete =====
